In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import seaborn as sns
import sys
import copy
sys.path.append("..")
from tqdm.notebook import tqdm
from numba import jit
from scipy import stats
import networkx as nx


import warnings
warnings.filterwarnings('ignore')
plt.style.use("../config/custom_plt.mplstyle")

colors = [
    "#7494d3",
    "#5cb545",
    "#9956c6",
    "#a7b338",
    "#6a6bc6",
    "#d09e40",
    "#ce62bb",
    "#56be85",
    "#d1477d",
    "#397f4d",
    "#cf4b4a",
    "#40bbc1",
    "#d8662c",
    "#99af66",
    "#b76989",
    "#6d7127",
    "#b6744a"
]

In [2]:
from sklearn.preprocessing import LabelEncoder
import networkx.algorithms.community as nx_comm

def load_results(inf_coords_path, labels, g):
    inf_coords = pd.read_csv(inf_coords_path, comment="#", header=None, sep="\s+")
    inf_coords.columns = ['index', 'kappa', 'hyp_rad', 'p1', 'p2', 'p3']
    inf_coords['index'] = inf_coords['index'].astype(str)
    inf_coords = inf_coords.merge(labels, on="index")
    le = LabelEncoder()
    inf_coords['encoded_label'] = le.fit_transform(inf_coords['label'])
    inf_coords = inf_coords.drop_duplicates(subset=['index'])
        
    # Louvain communities
    communities = nx_comm.louvain_communities(g, seed=123)
    communities_dict = []
    for i, com in enumerate(communities):
        communities_dict.append({c:i for c in com})

    result = {}
    for d in communities_dict:
        result.update(d)

    communities_louvain = pd.DataFrame()
    communities_louvain['index'] = result.keys()
    communities_louvain['label_louvain'] = result.values()

    inf_coords = inf_coords.merge(communities_louvain, on='index')        
    print('Number of communities from Louvain: ', len(np.unique(inf_coords['label_louvain'])))

    return inf_coords

In [3]:
imdb_graph = nx.read_edgelist("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/imdb/imdb_mam/eS1/imdb_mam_GC.edge")
imdb_edges = nx.to_pandas_edgelist(imdb_graph)
imdb_labels = np.loadtxt("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/imdb/imdb_label.txt")
imdb_labels = np.argmax(imdb_labels, axis=1)
imdb_labels_dict = pd.DataFrame()
imdb_labels_dict['label'] = imdb_labels
imdb_labels_dict = imdb_labels_dict.reset_index()
imdb_labels_dict['index'] = imdb_labels_dict['index'].astype(str)

base_path = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/imdb/umap/"
imdb_le_ml = load_results(f"{base_path}/le_ml/imdb_mam_GC.inf_coord", imdb_labels_dict, imdb_graph)
imdb_umap_ml = load_results(f"{base_path}/umap_ml/imdb_mam_GC.inf_coord", imdb_labels_dict, imdb_graph)
imdb_only_umap = load_results(f"{base_path}/only_umap/imdb_mam_GC.inf_coord", imdb_labels_dict, imdb_graph)

imdb_find_k = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/imdb_labels_umap_find_k_cC.csv")
imdb_find_k['index'] = imdb_find_k['index'].astype(str)

imdb_le_ml = imdb_le_ml.merge(imdb_find_k)
imdb_umap_ml = imdb_umap_ml.merge(imdb_find_k)
imdb_only_umap = imdb_only_umap.merge(imdb_find_k)


Number of communities from Louvain:  27
Number of communities from Louvain:  27
Number of communities from Louvain:  27


In [4]:
imdb_le_ml

,index,kappa,hyp_rad,p1,p2,p3,label,encoded_label,label_louvain,label_clustering_find_k
0,0,7.208790,12.0511,13.640800,-6.57720,5.24835,1,1,1,2
1,1,41.807900,10.2933,0.381966,4.82550,15.27890,1,1,5,2
2,2,9.338030,11.7923,5.796720,12.29230,-8.49545,1,1,8,2
3,3,30.093100,10.6221,14.263400,2.48076,6.87593,1,1,2,2
4,4,4.990290,12.4189,10.985600,7.22482,-9.16490,1,1,23,2
...,...,...,...,...,...,...,...,...,...,...
3223,3539,0.887547,14.1457,14.729600,5.91043,2.23173,2,2,3,4
3224,3540,3.165660,12.8740,14.175500,-5.54447,-5.01887,1,1,18,3
3225,3543,1.639570,13.5320,14.162600,7.25366,1.91879,0,0,7,3
3226,3545,2.598420,13.0715,13.386400,7.38562,4.80957,2,2,19,4


In [5]:
import pyvista as pv
pv.global_theme.color = 'white'

# Different color scheme for different type of labels


# For Metadata
# new_colors = ["#b18281", "#6d45cd", "#62a03b", "#c84ccb", "#a68b3c", "#482a79","#d74327", 
#              "#6f7dcf", "#cf783d", "#608aa4", "#cd4859", "#5f9c7b", "#d2478d", "#44532d", 
#              "#b773b5", "#703425", "#342d40", "#723057"]

# # For Topology
# new_colors = ["#dca083", "#7dbad3", "#983aa1", "#b1c232", "#db4393", "#a5662f", "#733fd4",
#               "#24192f", "#3d2560", "#4d8163", "#64c27a", "#a9ad84", "#86af46", "#db502c",
#               "#882d2a", "#50222f", "#d64ed7", "#c4a2c5", "#34475e", "#5a3d23", "#e19a2e",
#               "#bea44b", "#557f8f", "#d8405a", "#462b8d", "#6a6d2b", "#5ec0ab", "#2b3f28",
#               "#648ace", "#d67681", "#54c840", "#397c30", "#8f6c64", "#903767", "#d082cd",
#               "#715c8b", "#7070dc"]

# # For Features
new_colors = ["#b8934e", "#c44f39", "#819bb1", "#4d393d", "#69aa55", "#c65b94", "#7d4cba"]


def get_spherical_cap_structure_grid(b, opening_angle, R, color_idx, radius=1.0):
    # From: https://stackoverflow.com/a/45458451
    r = R
    phi = np.linspace(0, 2 * np.pi, 30)
    theta = np.linspace(0, opening_angle, 20)
    X = r * np.stack([
        np.outer(np.cos(phi), np.sin(theta)),
        np.outer(np.sin(phi), np.sin(theta)),
        np.outer(np.ones(np.size(phi)), np.cos(theta)),
        ], axis=-1)

    # rotate X such that [0, 0, 1] gets rotated to `c`;
    # <https://math.stackexchange.com/a/476311/36678>.
    a = np.array([0.0, 0.0, 1.0])
    a_x_b = np.cross(a, b)
    a_dot_b = np.dot(a, b)
    if a_dot_b == -1.0:
        X_rot = -X
    else:
        X_rot = (
            X +
            np.cross(a_x_b, X) +
            np.cross(a_x_b, np.cross(a_x_b, X)) / (1.0 + a_dot_b)
            )
        
    return pv.StructuredGrid(X_rot[..., 0], X_rot[..., 1], X_rot[..., 2])

In [6]:
def get_geodesic(p1, p2):
    omega = np.arccos(np.dot(p1, p2) / (np.linalg.norm(p1) * np.linalg.norm(p2)))
    t = np.linspace(0, 1)
    
    line = []
    for t in np.linspace(0, 1):
        line.append(np.sin((1 - t) * omega) / np.sin(omega) * p1 + np.sin(t * omega) / np.sin(omega) * p2)
    return np.array(line)

In [7]:
def compute_prob_S2(p1, p2, kappa1, kappa2):
    beta = 5.12
    mu = 0.0125
    R = 1
    angle = np.arccos(np.dot(p1, p2) / (np.linalg.norm(p1) * np.linalg.norm(p2)))
    
    chi = (R * angle) / np.sqrt(kappa1 * kappa2 * mu)
    return 1 / (1 + np.power(chi, beta))

In [8]:
def plot_embedding(df, label):
    pv.set_plot_theme("document")
    plotter = pv.Plotter(window_size=[4096, 4096])

    plotter.enable_anti_aliasing('ssaa')

    R = 1
    u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:100j]
    x = R*np.cos(u)*np.sin(v)
    y = R*np.sin(u)*np.sin(v)
    z = R*np.cos(v)
    grid = pv.StructuredGrid(x, y, z)
    plotter.add_mesh(grid, color='#fdfdfd', opacity=1)

    # Plot edges
    pos = df[['p1', 'p2', 'p3']].values
    pos /= np.linalg.norm(pos, axis=1)[:, None]
    kappa = df['kappa'].values

    count = 0
    for source, target in tqdm(imdb_edges.values):
        s_i = df['index'].tolist().index(source)
        t_i = df['index'].tolist().index(target)

        # Compute the probability of connection
        p1, p2 = pos[s_i], pos[t_i]
        prob = compute_prob_S2(p1, p2, kappa[s_i], kappa[t_i])
        if prob < 0.999: # filter out low probable links
            count += 1
            continue

        l = get_geodesic(p1, p2)
        actor = plotter.add_lines(l, color='#8a8a8a', width=6*prob)

    print('Number of low probable links: ', count)

    max_kappa = max(df['kappa'].values)
    idx = 0
    i = 0
    R = 1.001
    for name, group in df.groupby(label):

        pos = group[['p1', 'p2', 'p3']].values
        for j in range(len(group)):
            p = pos[j] / np.linalg.norm(pos[j])
            s = group['kappa'].values[j]
            s /= max_kappa
            
            s *= 0.1
            
            cap = get_spherical_cap_structure_grid(p, s, R, color_idx=idx)
            plotter.add_mesh(cap, color=new_colors[idx])
            i += 1
        idx += 1


    plotter.camera_position = 'yz'

    # LE+ML
    plotter.camera.azimuth = 330
    plotter.camera.elevation = 30
    
    # UMAP+ML
#     plotter.camera.azimuth = 20
#     plotter.camera.elevation = 30
    
    return plotter

In [9]:
pv.start_xvfb()

In [10]:
# plotter = plot_embedding(imdb_le_ml, 'label')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_le_ml_labels_metadata.jpg")


In [11]:
# plotter = plot_embedding(imdb_umap_ml, 'label')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_umap_ml_labels_metadata.jpg")


In [12]:
# plotter = plot_embedding(imdb_only_umap, 'label')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_only_umap_labels_metadata.jpg")


In [13]:
# plotter = plot_embedding(imdb_le_ml, 'label_louvain')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_le_ml_labels_topology.jpg")


In [14]:
# plotter = plot_embedding(imdb_umap_ml, 'label_louvain')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_umap_ml_labels_topology.jpg")


In [15]:
# plotter = plot_embedding(imdb_only_umap, 'label_louvain')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_only_umap_labels_topology.jpg")


In [16]:
# plotter = plot_embedding(imdb_le_ml, 'label_clustering_find_k')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_le_ml_labels_features.jpg")


In [17]:
# plotter = plot_embedding(imdb_umap_ml, 'label_clustering_find_k')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_umap_ml_labels_features.jpg")


In [18]:
# plotter = plot_embedding(imdb_only_umap, 'label_clustering_find_k')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/imdb_only_umap_labels_features.jpg")


  0%|          | 0/31401 [00:00<?, ?it/s]

Number of low probable links:  23291


array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]